# 3. 단어 추출 word extraction

## 3-a. 단어 추출을 하는 이유
- 커뮤니티 text는 신조어, 은어가 많다.
- 토큰화를 하기 위해선, 신조어와 은어를 추출해야 한다.

## 3-b. 단어 추출 계획
1. text로만 이루어진 txt 파일 만들기
2. soynlp 의 WordExtractor 를 이용하여 단어 추출
3. 단어와 수치 데이터를 csv 파일로 저장

## 3-c. 단어 추출 진행

### 3-c-1. 파일 불러오기

In [1]:
import pandas as pd
df = pd.read_csv("2_spaced_annlab_text.csv", encoding="utf-8")
df.head(3)

,community,gall_id,search_keyword,number,date_created,time_created,author,is_reply,spaced_text
0,dcinside,snp500,안랩,806331,2023-09-08,13:01:47,서울에핵한방,0,매크로 안보이네 국장으로 대피한다 공습경보 공습경보안랩과 써니전자를 매수하라
1,dcinside,snp500,안랩,732959,2023-05-20,23:40:40,ㅇㅇ(121.131),0,한국 주식중에서 년내 수익률 는 먹는거 안랩 내년 총선 앞두고 안철수 나서면서 이것...
2,dcinside,snp500,안랩,732959,2023-05-20,23:47:45,ㅇㅇ(121.131),1,비추 왜 누름?이런거 사놓고 존버하면 먹는거 맞는데 ㅋ


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28952 entries, 0 to 28951
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   community       28952 non-null  object
 1   gall_id         28952 non-null  object
 2   search_keyword  28952 non-null  object
 3   number          28952 non-null  int64 
 4   date_created    28952 non-null  object
 5   time_created    28952 non-null  object
 6   author          28952 non-null  object
 7   is_reply        28952 non-null  int64 
 8   spaced_text     28952 non-null  object
dtypes: int64(2), object(7)
memory usage: 2.0+ MB


### 3-c-2. corpus를 list 형태로 만든다

In [3]:
corpus = df['spaced_text'].tolist()

### 3-c-3. word_extractor 학습시키기

In [4]:
from soynlp.word import WordExtractor

word_extractor = WordExtractor(min_frequency=100,
    min_cohesion_forward=0.05, 
    min_right_branching_entropy=0.0
)
word_extractor.train(corpus) # list of str or like
words = word_extractor.extract()

training was done. used memory 0.303 Gbory 0.148 Gb
all cohesion probabilities was computed. # words = 1490
all branching entropies was computed # words = 57684
all accessor variety was computed # words = 57684


In [5]:
words['주식']

Scores(cohesion_forward=0.39057377049180325, cohesion_backward=0, left_branching_entropy=3.997724532768562, right_branching_entropy=3.8157065719273593, left_accessor_variety=143, right_accessor_variety=110, leftside_frequency=1906, rightside_frequency=0)

In [6]:
len(words)

1410

### 3-c-4. word extraction 결과 출력

In [10]:
# 결과 출력

import math

def word_score(score):
    return (score.cohesion_forward * math.exp(score.right_branching_entropy))

print('단어   (빈도수, cohesion, branching entropy, word_score)\n')
for word, score in sorted(words.items(), key=lambda x:word_score(x[1]), reverse=True):
    print('%s     (%d, %.3f, %.3f, %.3f)' % (
            word, 
            score.leftside_frequency, 
            score.cohesion_forward,
            score.right_branching_entropy,
            word_score(score)
            )
         )

단어   (빈도수, cohesion, branching entropy, word_score)

ㅋㅋㅋ     (3471, 0.842, 4.546, 79.340)
철수     (1435, 0.903, 4.306, 66.917)
안랩     (12178, 0.512, 4.666, 54.427)
철수가     (351, 0.470, 4.619, 47.647)
너무     (592, 0.658, 4.265, 46.832)
진짜     (866, 0.474, 4.532, 44.043)
테마주     (818, 0.811, 3.910, 40.487)
안철수가     (1151, 0.364, 4.533, 33.902)
존나     (486, 0.561, 4.099, 33.802)
ㅋㅋ     (4831, 0.986, 3.515, 33.162)
때문에     (383, 0.598, 4.014, 33.133)
ㄹㅇ     (507, 0.964, 3.516, 32.434)
철수는     (288, 0.426, 4.243, 29.629)
된다     (278, 0.792, 3.607, 29.204)
근데     (681, 0.711, 3.713, 29.124)
계속     (460, 0.529, 3.987, 28.524)
에서     (100, 0.141, 5.148, 24.335)
안철수     (7186, 0.550, 3.719, 22.677)
찰스     (882, 0.792, 3.336, 22.254)
종목     (497, 0.511, 3.754, 21.797)
민주당     (399, 0.664, 3.490, 21.757)
관련주     (333, 0.466, 3.819, 21.231)
까지     (105, 0.187, 4.705, 20.690)
라고     (231, 0.407, 3.919, 20.514)
단일화     (521, 0.627, 3.487, 20.490)
ㅠㅠ     (389, 0.917, 3.085, 20.065)
같은     (858, 0.425,

### 3-c-5. threshold 판단
- 눈으로 직접 살펴보며, 무의미한 결과를 거를 word_score의 threshold를 정한다
- 살펴본 결과. word_score >= 0.230 인 값만 유의미한 데이터로 간주한다
- 결과 dict : scored_words

In [11]:
threshold = 0.230
# threshold 를 기준으로 dictionary 생성
scored_words = {word: word_score(score) for word, score in words.items() if word_score(score) >= threshold}

print(scored_words)

{'엄청': 8.704903217316591, '질문': 0.5, '누구': 2.1014025053386063, '월에': 2.2497464242105942, '나도': 3.5684298105312657, '것도': 3.8191290740687274, '된다': 29.203656441115694, '몰빵': 3.3281672793759087, '까지': 20.68981656381017, '떡락': 1.382114324140229, 'lg': 1.7098973071418857, '지금': 14.19197587435067, '대한': 1.411737560074441, '등등': 2.8111303611480714, '돌아': 0.39346811819595645, '되는': 8.386047757041137, '학교': 3.7399528613996726, '미래': 1.3934410914682969, '구성': 0.5912678963748776, 'bw': 4.116217365065151, '팔아': 0.36527806425838744, '그래': 0.24886805808159107, '발표': 0.6033695816170062, '라는': 16.331288094564574, '책임': 0.8773826753016616, '똑같': 0.7045454545454546, '박근': 0.3142358304736506, '리더': 0.3522388059701492, '반대': 0.3523419196694001, '영향': 1.2591626909511189, '뛰어': 0.601063829787234, '다음': 1.4807402832058394, 'ㅅㅂ': 16.90368452304342, '관련': 2.2410837670437243, '발행': 1.4736619676064109, '기사': 0.7126659439661441, '결정': 0.5186766275346851, '바이': 0.40185914240804266, '어떤': 2.521385464759107, '저거': 

In [12]:
len(scored_words)

738

## 3-d. 단어 정제
1. 영어 포함된 단어 정제
  - 특수한 경우이므로, 프로세스를 나누어 진행한다
2. 조사 포함된 단어 제거
  - "sk텔레콤에" 처럼 조사가 붙은 단어들을 제거한다.
  - 단어 끝에 붙은 조사 처리 : "은", "는", "으로", "이다", "합니다", "였다", "에", "와", "과"
  - list를 뽑아보고, 예외로 처리할 게 있는지 판단한다.
  - "이/가"는 "시가" 같은 멀쩡한 단어를 쪼갤 수 있으니, 진행하지 않거나 주의한다.
    

### 3-d-1. 단어 정제 : 영어가 포함된 단어
- 안랩에서는 영어 전부 제거

In [14]:
import re

def contains_english(key):
    # 키 값에 영어 알파벳이 하나라도 포함되어 있는지 확인
    return bool(re.search(r'[a-zA-Z]', key))

# 영어 알파벳이 포함된 키들만 추출
english_included_keys = [key for key in scored_words.keys() if contains_english(key)]

print(english_included_keys)
# for key, value in scored_words.items():
#     if contains_english(key):
#         print(f"{key}: {value}")

['lg', 'bw', 'co', 'k씨', 'it', 'jp', 'mb', 'vs', 'sk', 'ht', 'ce', 'htt', 'com', 'ceo', 'http']


In [16]:
len(english_included_keys)

15

In [17]:
# scored_words 에 적용
def remove_matching_keys(dictionary, key_list):
    return {k: v for k, v in dictionary.items() if k not in key_list}

# 함수 적용
scored_words_2 = remove_matching_keys(scored_words, english_included_keys)
len(scored_words_2)

723

### 3-d-2. 단어 정제 : 조사가 포함된 단어

In [19]:
key_list

['엄청',
 '질문',
 '누구',
 '월에',
 '나도',
 '것도',
 '된다',
 '몰빵',
 '까지',
 '떡락',
 '지금',
 '대한',
 '등등',
 '돌아',
 '되는',
 '학교',
 '미래',
 '구성',
 '팔아',
 '그래',
 '발표',
 '라는',
 '책임',
 '똑같',
 '박근',
 '리더',
 '반대',
 '영향',
 '뛰어',
 '다음',
 'ㅅㅂ',
 '관련',
 '발행',
 '기사',
 '결정',
 '바이',
 '어떤',
 '저거',
 '컴퓨',
 '경우',
 '선언',
 '본다',
 '올해',
 '네이',
 '코스',
 '퍼포',
 '지난',
 '애들',
 'ㅇㅇ',
 '같이',
 '삼전',
 '하지',
 '아닌',
 '국힘',
 '외국',
 '카이',
 '에서',
 '크게',
 '먼저',
 '에스',
 '타이',
 '포기',
 '팩트',
 '투자',
 '보수',
 '최소',
 '들고',
 '고점',
 '최대',
 '단타',
 '제발',
 '자기',
 '몰라',
 'ㄷㄷ',
 '너무',
 '수도',
 '하고',
 '카카',
 '대표',
 '먹고',
 '요즘',
 '여러',
 '미친',
 '당선',
 '철수',
 '역시',
 '작년',
 '세계',
 '연락',
 '일이',
 '감사',
 '신고',
 '차트',
 '괜찮',
 '경영',
 '확인',
 '캬ㅋ',
 '중국',
 '존버',
 '앞으',
 '졸업',
 '느낌',
 '준비',
 '정치',
 '돈이',
 '총선',
 '만원',
 '많아',
 '까뮤',
 '시간',
 '개미',
 '언제',
 '오른',
 '단일',
 '거기',
 '미국',
 '합당',
 '갤러',
 '원래',
 '가장',
 '직접',
 '오를',
 '이번',
 '우리',
 '하나',
 '쓰는',
 '이런',
 '추천',
 '호남',
 '직원',
 '셀트',
 '솔직',
 '일단',
 'ㅠㅠ',
 '부분',
 '의원',
 '상장',
 '국내',
 '백신',
 '방법',
 '본인',
 '젊은',
 '윤석',

In [18]:
key_list = list(scored_words_2.keys())
len(key_list)

723

#### 조사 필터에 추가할 단어 고르기
- 조사 하나하나 테스트해본다
- "은/는", "으로", "이다", "합니다", "였다", "에", "와/과", "이/가"
- 결과 : scored_words_3

In [51]:
# 특정 문자열로 끝나는 단어 리턴
def find_endswith(input_list, endswith):
    result = [item for item in input_list if item.endswith(endswith)]
    return result
find_endswith(key_list, "자체")

[]

In [53]:
list_1 = ['철수는', '안철수는', '테마주는', '안랩은', '발언은', '사람은', '원장은', '지금은', '주식은', '사람들은']
list_2 = ['안랩으로', '퍼포먼스식으로', '안랩에', '당시에', '이번에', '때문에', '대선에', '만원에', '안철수에']
list_3 = ['원장과', '지금이', '주식이', '원장이', '호남이', '본인이', '사람이', '때문이', '안랩이', '자신이', 
          '생각이', '국민들이', '사람들이', '대통령이', '가능성이', '정치인이', '문제가', '이유가', '주가가', 
          '후보가', '철수가', '찰스가', '안철수가']

#### 조사 필터에 추가

In [54]:
particle_filter = []
particle_filter.extend(list_1)
particle_filter.extend(list_2)
particle_filter.extend(list_3)
particle_filter

['철수는',
 '안철수는',
 '테마주는',
 '안랩은',
 '발언은',
 '사람은',
 '원장은',
 '지금은',
 '주식은',
 '사람들은',
 '안랩으로',
 '퍼포먼스식으로',
 '안랩에',
 '당시에',
 '이번에',
 '때문에',
 '대선에',
 '만원에',
 '안철수에',
 '원장과',
 '지금이',
 '주식이',
 '원장이',
 '호남이',
 '본인이',
 '사람이',
 '때문이',
 '안랩이',
 '자신이',
 '생각이',
 '국민들이',
 '사람들이',
 '대통령이',
 '가능성이',
 '정치인이',
 '문제가',
 '이유가',
 '주가가',
 '후보가',
 '철수가',
 '찰스가',
 '안철수가']

#### 기존 단어 목록에 조사 필터 적용하기

In [55]:
# 함수 적용
scored_words_3 = remove_matching_keys(scored_words_2, particle_filter)
len(scored_words_3)

681

In [56]:
scored_words_3

{'엄청': 8.704903217316591,
 '질문': 0.5,
 '누구': 2.1014025053386063,
 '월에': 2.2497464242105942,
 '나도': 3.5684298105312657,
 '것도': 3.8191290740687274,
 '된다': 29.203656441115694,
 '몰빵': 3.3281672793759087,
 '까지': 20.68981656381017,
 '떡락': 1.382114324140229,
 '지금': 14.19197587435067,
 '대한': 1.411737560074441,
 '등등': 2.8111303611480714,
 '돌아': 0.39346811819595645,
 '되는': 8.386047757041137,
 '학교': 3.7399528613996726,
 '미래': 1.3934410914682969,
 '구성': 0.5912678963748776,
 '팔아': 0.36527806425838744,
 '그래': 0.24886805808159107,
 '발표': 0.6033695816170062,
 '라는': 16.331288094564574,
 '책임': 0.8773826753016616,
 '똑같': 0.7045454545454546,
 '박근': 0.3142358304736506,
 '리더': 0.3522388059701492,
 '반대': 0.3523419196694001,
 '영향': 1.2591626909511189,
 '뛰어': 0.601063829787234,
 '다음': 1.4807402832058394,
 'ㅅㅂ': 16.90368452304342,
 '관련': 2.2410837670437243,
 '발행': 1.4736619676064109,
 '기사': 0.7126659439661441,
 '결정': 0.5186766275346851,
 '바이': 0.40185914240804266,
 '어떤': 2.521385464759107,
 '저거': 2.047241352371

### 3-d-3. 단어 정제 : 육안으로 판별하기
- 결과 : filtered_scored_words

In [58]:
my_filter = [
    '대한', '것도', '돌아', '되는', '갑자', '박근', '바이', '컴퓨', '네이', '코스', '퍼포', '하지',
    '아닌', '카이', '에스', '타이', '들고', '자기', '하고', '카카', '일이', '괜찮', '캬ㅋ', '앞으', 
    '돈이', '많아', '까뮤', '오른', '단일', '거기', '합당', '갤러', '오를', '쓰는', '솔직', '윤석', 
    '알아', '위한', '적이', '살아', '믿고', '나가', '못한', '한다', '아님', '하는', '받았', '팔고', 
    '있는', '케이', '했던', '상한', '해서', '했다', '되면', '움직', '아무', '아니', '걸어', '클라', 
    '편이', '것은', '말을', '봐도', '갖고', '있음', '없어', '빠지', '달라', '것이', '떨어', '먹는', 
    '있다', '알고', '나올', '밀어', '힘들', '받아', '겁니', '끌어', '합니', '못하', '년에', '남선', 
    '맞는', '끝나', '나오', '없는', '것을', '있어', '벌었', '황교', '홍준', '안철', '않는', '들어', 
    '않았', '어디', '문재', '같아', '풀매', '그런', '나온', '모르', '넘어', '않은', '해야', '없고', 
    '모르겠', '말했다', '대선때', '여론조', '안되는', '겁니다', '자신의', '안랩도', '얘기를', '써니',
    '있다는', '사이에', '퍼포먼', '우리나', '기업의', '서울의', '들어가', '있으면', '얘기다', 
    '바이러', '생각하', '캬ㅋㅋ', '좋아하', '팔아야', '안랩을', '인증해', '안랩의', '것이다', 
    '국민의', '합니다', '자체가', '백지신', '한다고', '못하는', '있는거', '들어갔', '올라가', 
    '정도로', '만들어', '있으니', '생각해', '받았다', '프로그', '가지고', '없는데', '카이스', 
    '있습니', '있다고', '주식을', '지지를', '하는거', '그러니', '말하는', '원장의', '써니전', 
    '그렇다', '떨어지', '않았다', '없어서', '있어서', '캬ㅋㅋㅋ', '인증해랑', '안랩에서', 
    '없습니다', '있습니다', '안철수의', '안철수에게', '퍼포먼스식으', '그만두겠', '그만두겠습니', 
]

# 함수 적용
filtered_scored_words = remove_matching_keys(scored_words_3, my_filter)
len(filtered_scored_words)

503

## 3-e. 단어 리스트를 파일로 저장

### 3-e-1. 파일로 저장하기

In [59]:
# 파일로 저장
filename = "3_annlab_word_list.txt"
words = filtered_scored_words
with open(filename, "w", encoding="utf-8") as file:
    for word in words:
        file.write(word + "\n")

print("파일 저장 완료.")

파일 저장 완료.


### 3-e-2. 불러와서 확인하기

In [60]:
# 불러올 파일명 지정
filename = "3_annlab_word_list.txt"

# 파일로부터 문자열 리스트 불러오기
with open(filename, "r", encoding="utf-8") as file:
    word_list = [line.rstrip('\n') for line in file]

print("불러온 문자열 리스트:")
print(word_list)

불러온 문자열 리스트:
['엄청', '질문', '누구', '월에', '나도', '된다', '몰빵', '까지', '떡락', '지금', '등등', '학교', '미래', '구성', '팔아', '그래', '발표', '라는', '책임', '똑같', '리더', '반대', '영향', '뛰어', '다음', 'ㅅㅂ', '관련', '발행', '기사', '결정', '어떤', '저거', '경우', '선언', '본다', '올해', '지난', '애들', 'ㅇㅇ', '같이', '삼전', '국힘', '외국', '에서', '크게', '먼저', '포기', '팩트', '투자', '보수', '최소', '고점', '최대', '단타', '제발', '몰라', 'ㄷㄷ', '너무', '수도', '대표', '먹고', '요즘', '여러', '미친', '당선', '철수', '역시', '작년', '세계', '연락', '감사', '신고', '차트', '경영', '확인', '중국', '존버', '졸업', '느낌', '준비', '정치', '총선', '만원', '시간', '개미', '언제', '미국', '원래', '가장', '직접', '이번', '우리', '하나', '이런', '추천', '호남', '직원', '셀트', '일단', 'ㅠㅠ', '부분', '의원', '상장', '국내', '백신', '방법', '본인', '젊은', '허허', '다른', '벤처', '간다', '없음', '의견', '추가', '친구', '재산', '경쟁', '지지', '좋아', '에이', '저런', '시장', '폭등', '자신', '현대', '밖에', '게임', '인간', '총리', '좋은', '일부', '그냥', '후보', '시작', '전부', '관심', '양보', '함락', '오늘', '처음', '분명', '되어', '보안', '때문', '뭐냐', '의사', '최근', '투표', '기술', '가치', '시총', '노조', '상승', '경제', '모든', '내가', '호재', '재료', '같다', '시절', '서울', '위해', '갤주', '발

## 3-f. 함수화
- 함수로 만들어 재사용 가능하도록 만든다.